In [4]:
import pandas as pd
import sqlalchemy
from sqlalchemy.orm import sessionmaker

url = "postgres://events_writer@localhost:5432/events"
readonly_user = "events_reader"
db = sqlalchemy.create_engine(url)

filename = "../datasets/icews/events.1995.csv"
df = pd.read_csv(filename, index_col=0, encoding="utf-8", parse_dates = ['EventDate'])

table_name = "icews"

def primary_key(df):
    return ','.join('"{0}"'.format(i) for i in df.index.names)
    
def table_exists(engine, table_name):
    return engine.dialect.has_table(engine, table_name)
    
if not table_exists(db, table_name):
    Session = sessionmaker(bind=db, autocommit=True)
    session = Session()

    df_empty = df.drop(df.index)
    df_empty.to_sql(table_name, db)
    
    with session.begin():
        session.execute('ALTER TABLE %s ADD PRIMARY KEY (%s);' % (table_name, primary_key(df)))
        session.execute('GRANT SELECT ON %s TO %s;' % (table_name, readonly_user))
    
df_existing = pd.read_sql('SELECT %s from %s' % (primary_key(df), table_name), db)
df_existing = df_existing.set_index(df.index.names)
df_merged = pd.merge(df, df_existing, how="inner", left_index=True, right_index=True, copy=False)

df_new = df.drop(df_merged.index)  

start = 0
end = 5

df_new[start:end].to_sql(table_name, db, if_exists = 'append')

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.orm import sessionmaker

url = "postgres://events_reader@localhost:5432/events"
db = sqlalchemy.create_engine(url)

filename = "../datasets/icews/events.1995.csv"
df = pd.read_csv(filename, index_col=0, encoding="utf-8", parse_dates = ['EventDate'])

table_name = "icews_test"

def primary_key(df):
    return ','.join('"{0}"'.format(i) for i in df.index.names)
    
def table_exists(engine, table_name):
    return engine.dialect.has_table(engine, table_name)
    
if not table_exists(db, table_name):
    Session = sessionmaker(bind=db, autocommit=True)
    session = Session()

    df_empty = df.drop(df.index)
    df_empty.to_sql(table_name, db)
    
    with session.begin():
        session.execute('ALTER TABLE %s ADD PRIMARY KEY (%s);' % (table_name, primary_key(df)))
    
df_existing = pd.read_sql('SELECT %s from %s' % (primary_key(df), table_name), db)
df_existing = df_existing.set_index(df.index.names)
df_merged = pd.merge(df, df_existing, how="inner", left_index=True, right_index=True, copy=False)

df_new = df.drop(df_merged.index)  

start = 0
end = 5

df_new[start:end].to_sql(table_name, db, if_exists = 'append')